In [3]:
import numpy as np
import keras
import tensorflow as tf
import keras.backend as K

In [13]:
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Input, GlobalAveragePooling2D, AveragePooling2D, Flatten, concatenate
from keras.models import Sequential, Model
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
import cv2

In [5]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols):
  (X_train, y_train), (X_test, y_test) = cifar10.load_data()

  # resize
  X_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in X_train])
  X_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in X_test])

  y_train = np_utils.to_categorical(y_train, num_classes=num_classes)
  y_test = np_utils.to_categorical(y_test, num_classes=num_classes)

  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  X_train = X_train/255
  X_test = X_test/255

  return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = load_cifar10_data(32, 32)

In [31]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_max_pool, name = None):

  conv_1x1 = Conv2D(filters_1x1, (1, 1), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(x)

  conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(x)
  conv_3x3 = Conv2D(filters_3x3, (3, 3), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(conv_3x3)

  conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(x)
  conv_5x5 = Conv2D(filters_5x5, (5, 5), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(conv_5x5)

  max_pool = MaxPool2D((3 ,3), strides=(1, 1), padding = 'same')(x)
  max_pool = Conv2D(filters_max_pool, (1, 1), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(max_pool)

  output = concatenate([conv_1x1, conv_3x3, conv_5x5, max_pool], axis = -1, name = None)

  return output

In [32]:
kernel_init = keras.initializers.glorot_uniform()

In [35]:
def build_model():
  inputs = Input((224, 224, 3))

  x = Conv2D(64, (7, 7), strides=(2, 2), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(inputs)
  x = MaxPool2D((3, 3), (2, 2), padding='same')(x)

  x = Conv2D(64, (1, 1), strides = (1, 1), padding  = 'same', activation = 'relu', kernel_initializer=kernel_init)(x)

  x = Conv2D(192, (3, 3), strides = (1, 1), padding = 'same', activation = 'relu', kernel_initializer=kernel_init)(x)
  x = MaxPool2D((3, 3), (2, 2), padding='same')(x)

  x = inception_module(x,
                       filters_1x1 = 64,
                       filters_3x3_reduce= 96,
                       filters_3x3 = 128,
                       filters_5x5_reduce = 16,
                       filters_5x5 = 32,
                       filters_max_pool = 32,
                       name = 'inception_3a')

  x = inception_module(x,
                       filters_1x1 = 128,
                       filters_3x3_reduce= 128,
                       filters_3x3 = 192,
                       filters_5x5_reduce = 32,
                       filters_5x5 = 96,
                       filters_max_pool = 64,
                       name = 'inception_3b')

  x = MaxPool2D((3, 3), (2, 2), padding='same')(x)

  x = inception_module(x,
                       filters_1x1 = 192,
                       filters_3x3_reduce= 96,
                       filters_3x3 = 208,
                       filters_5x5_reduce = 16,
                       filters_5x5 = 48,
                       filters_max_pool = 64,
                       name = 'inception_4a')

  x1 = AveragePooling2D((5, 5), strides=3)(x)
  x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
  x1 = Flatten()(x1)
  x1 = Dense(1024, activation='relu')(x1)
  x1 = Dropout(0.7)(x1)
  x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

  x = inception_module(x,
                       filters_1x1 = 160,
                       filters_3x3_reduce= 112,
                       filters_3x3 = 224,
                       filters_5x5_reduce = 24,
                       filters_5x5 = 64,
                       filters_max_pool = 64,
                       name = 'inception_4b')

  x = inception_module(x,
                       filters_1x1 = 128,
                       filters_3x3_reduce= 128,
                       filters_3x3 = 256,
                       filters_5x5_reduce = 24,
                       filters_5x5 = 64,
                       filters_max_pool = 64,
                       name = 'inception_4c')

  x = inception_module(x,
                       filters_1x1 = 112,
                       filters_3x3_reduce= 144,
                       filters_3x3 = 288,
                       filters_5x5_reduce = 32,
                       filters_5x5 = 64,
                       filters_max_pool = 64,
                       name = 'inception_4d')

  x2 = AveragePooling2D((5, 5), strides=3)(x)
  x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
  x2 = Flatten()(x2)
  x2 = Dense(1024, activation='relu')(x2)
  x2 = Dropout(0.7)(x2)
  x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

  x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_max_pool=128,
                     name='inception_4e')
  
  x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

  x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_max_pool=128,
                     name='inception_5a')

  x = inception_module(x,
                      filters_1x1=384,
                      filters_3x3_reduce=192,
                      filters_3x3=384,
                      filters_5x5_reduce=48,
                      filters_5x5=128,
                      filters_max_pool=128,
                      name='inception_5b')

  x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

  x = Dropout(0.4)(x)

  x = Dense(10, activation='softmax', name='output')(x)

  model = Model(inputs, [x, x1, x2], name='inception_v1')

  return model

In [36]:
model = build_model()
model.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 112, 112, 64) 9472        input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 56, 56, 64)   4160        max_pooling2d_11[0][0]           
_______________________________________________________________________________________

In [40]:
import math

initial_lrate = 0.01
epochs = 25

def decay(epoch, steps = 100):
  initial_lrate = 0.01
  drop = 0.96
  epochs_drop = 8
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

sgd = keras.optimizers.SGD(learning_rate = initial_lrate, momentum = 0.9, nesterov = False)
lrsc = LearningRateScheduler(decay, verbose = 1)

model.compile(loss = ['categorical_crossentropy','categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics = ['accuracy'])
  

In [ ]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])